In [1]:
import pycrunchbase

In [2]:
from operator import itemgetter
import re

In [3]:
# CONSTANTS
API_KEY = 'd8c293682411de52d3c5280410fd2419'

In [4]:
cb = pycrunchbase.CrunchBase(API_KEY)

In [5]:
% ls ids_data/

funding_rounds.csv  ipos.csv            people.csv
funds.csv           organizations.csv   products.csv


## Funding Rounds

In [7]:
PROPERTIES = [
    u'funding_type',
    u'money_raised_usd',
    u'series',
    u'announced_on',
    u'announced_on_trust_code',
    u'closed_on',
    u'closed_on_trust_code',
]

RELATIONSHIPS_FUNDED_ORGANIZATION = [
    u'uuid',
]

In [8]:
output_file = file('data/funding_rounds.csv','w')
output_file.write('uuid,')
[output_file.write(prop+",") for prop in PROPERTIES]
output_file.write('org_uuid')

In [9]:
def get_fund_round_string(funding_round):
    properties_dict = funding_round.data['properties']
    organization_dict = funding_round.data['relationships']['funded_organization']['item']
    properties_values = itemgetter(*PROPERTIES)(properties_dict)
    organization_values = itemgetter(*RELATIONSHIPS_FUNDED_ORGANIZATION)(organization_dict)
    return "\n"+str(funding_round.data['uuid'])+"," + ",".join([str(x) for x in properties_values])+","+ organization_values


In [10]:
input_file = open('ids_data/funding_rounds.csv')
input_file.readline()

'uuid,updated_at\n'

In [ ]:
for line in input_file:
    uuid = line.split(",")[0]
    try:
        funding_round = cb.funding_round(str(uuid))
        output_file.write(get_fund_round_string(funding_round))
    except:
        print "Failed to get the Funding Round for ", uuid

Failed to get the Funding Round for  0115002a-760a-8974-7ba0-ab2665b95461


In [ ]:
output_file.close()
input_file.close()

## Organizations

### Create Files

#### Addresses

In [ ]:
address_file = open('data/address.csv','w')
address_file.write('org_uuid,city,region,country,latitude,longitude,type\n')
address_file.close()

In [522]:
def store_offices(org):
    [store_address(org.uuid, address) for address in org.offices.items]
    
def store_headquarters(org):
    if org.data['relationships']['headquarters']['paging']['total_items']>0:
        store_address(org.uuid,org.headquarters, True)
    
def store_address(uuid,address, headquarters= False):
    address_file = open('data/address.csv','a')
    address_file.write(org.uuid+","+address.city+ ", "+ address.region+","+address.country+","+str(address.latitude)+","+str(address.longitude)+","+str(headquarters)+"\n")
    address_file.close()

#### Competitors

In [ ]:
competitors_file =  open('data/competitors.csv','w')
competitors_file.write('org_uuid,competitor_uuid')
competitors_file.close()

In [525]:
def store_competitors_details(org):
    competitors_file = open('data/competitors.csv','a')
    competitors = cb.more(org.competitors)
    [competitors_file.write(org.uuid+","+competitor.uuid+"\n") for competitor in competitors]
    competitors_file.close()

#### Acquisitions

In [487]:
acquisitions_file = open('data/acquisitions.csv','w')
acquisitions_file.write(
'acquirer_uuid,\
acquiree_uuid,\
price,\
price_currency_code,\
price_usd,\
payment_type,\
acquisition_type,\
acquisition_status,\
disposition_of_acquired,\
announced_on,\
announced_on_trust_code,\
completed_on\n'
)
acquisitions_file.close()

In [500]:
def store_acquisition(org):
    acquisitions = org.acquisitions
    acquisitions_file = open('data/acquisitions.csv','a')
    [acquisitions_file.write(
        str(acq.uuid)+","+\
        str(acq.acquiree.get(0).uuid)+","+\
        str(acq.price)+","+\
        str(acq.price_currency_code)+","+\
        str(acq.price_usd)+","+\
        str(acq.payment_type)+","+\
        str(acq.acquisition_type)+","+\
        str(acq.acquisition_status)+","+\
        str(acq.disposition_of_acquired)+","+\
        str(acq.announced_on)+","+\
        str(acq.announced_on_trust_code)+","+\
        str(acq.completed_on)+"\n"
    ) for acq in acquisitions]

#### Organization Properties

In [471]:
organizations_file = open('data/organizations.csv','w')
organizations_file.write(
'org_uuid,\
closed_on,\
closed_on_trust_code,\
founded_on,\
founded_on_trust_code,\
is_closed,\
name,\
num_employees_max,\
num_employees_min,\
number_of_investments,\
stock_exchange,\
stock_symbol,\
total_funding_usd,\
created_at,\
updated_at,\
role_company,\
role_group,\
role_investor,\
role_school\n'
)
organizations_file.close()

In [470]:
def store_organization_properties(org):
    organizations_file = open('data/organizations.csv','a')
    organizations_file.write(
        org.uuid+","\
        +str(org.closed_on)+","\
        +str(org.closed_on_trust_code)+","\
        +str(org.founded_on)+","\
        +str(org.founded_on_trust_code)+","\
        +str(org.is_closed)+","\
        +str(org.name)+","\
        +str(org.num_employees_max)+","\
        +str(org.num_employees_min)+","\
        +str(org.number_of_investments)+","\
        +str(org.stock_exchange)+","\
        +str(org.stock_symbol)+","\
        +str(org.total_funding_usd)+","\
        +str(org.created_at)+","\
        +str(org.updated_at)+","\
        +str(org.role_company)+","\
        +str(org.role_group)+","\
        +str(org.role_investor)+","\
        +str(org.role_school)+"\n"
    )
    organizations_file.close()

#### Founders

In [599]:
founders_file = open('data/founders.csv','w')
founders_file.write('org_uuid, founder_uuid\n')
founders_file.close()

In [598]:
def store_founders(org):
    founders_file = open('data/founders.csv','a')
    for founder in org.founders:
        founders_file.write(org.uuid+","+founder.uuid+"\n")
    founders_file.close()

### Read all organizations

In [502]:
input_file = open('ids_data/organizations.csv')
input_file.readline()

'name,permalink,updated_at\n'

In [503]:
pattern = re.compile('.*,/organizations/(.*),.*')

In [504]:
def store_organization(org):
    store_headquarters(org)
    store_offices(org)
    store_competitors_details(org)
    store_organization_properties(org)

In [600]:
count = 0
for line in input_file:
    
    permalink =  pattern.match(line).group(1)
    print permalink
    org = cb.organization(permalink)
    store_founders(org)
    if count >10:
        break
    else:
        count+=1

aarsh-infotech
the-law-offices-of-david-b-shapiro
het-zorghorloge
the-pennsylvania-state-university
the-game-crafter
ironworks-consulting
la-sirena
bistro
saic-capital
mirchee
paintbox-catering-and-bistro-inc
eligible-api


In [337]:
PROPERTIES = [
    'closed_on',
    'closed_on_trust_code',
    'founded_on',
    'founded_on_trust_code',
    'is_closed',
    'name',
    'num_employees_max',
    'num_employees_min',
    'number_of_investments',
    'stock_exchange',
    'stock_symbol',
    'total_funding_usd',
    'created_at',
    'updated_at',
    'role_company',
    'role_group',
    'role_investor',
    'role_school',
]

RELATIONSHIP_ADDRESS = [
    'city',
    'region',
    'country',
    'latitude',
    'longitude',
]

RELATIONSHIP_ACQUISITION = [
    'price',
    'price_currency_code',
    'price_usd',
    'payment_type',
    'acquisition_type',
    'acquisition_status',
    'disposition_of_acquired',
    'announced_on',
    'announced_on_trust_code',
    'completed_on',
]

## People 

### Create Files

#### Degrees

In [692]:
degrees_file = open('data/degrees.csv', 'w')
degrees_file.write(
'uuid,\
person_uuid,\
school_uuid,\
started_on,\
started_on_trust_code,\
is_completed,\
completed_on,\
completed_on_trust_code,\
degree_type_name,\
degree_subject,\
created_at,\
updated_at'
)
degrees_file.close()

In [680]:
def store_degrees(person):
    degrees_file = open('data/degrees.csv', 'a')
    for degree in person.degrees:
        degrees_file.write(
            degree.uuid+","+\
            person.uuid+","+\
            degree.school[0].uuid+","+\
            str(degree.started_on)+","+\
            str(degree.started_on_trust_code)+","+\
            str(degree.is_completed)+","+\
            str(degree.completed_on)+","+\
            str(degree.completed_on_trust_code)+","+\
            str(degree.degree_type_name)+","+\
            str(degree.degree_subject)+","+\
            str(degree.created_at)+","+\
            str(degree.updated_at)+"\n"
        )
    degrees_file.close()

#### Jobs

In [679]:
jobs_file = open('data/jobs.csv','w')
jobs_file.write(
'ended_on,\
ended_on_trust_code,\
started_on,\
started_on_trust_code,\
title,\
uuid,\
person_uuid,\
org_uuid\n'
)
jobs_file.close()

In [689]:
def store_jobs(person):
    jobs_file = open('data/jobs.csv','a')
    for job in person.jobs:
        jobs_file.write(
            str(job.ended_on)+","+\
            str(job.ended_on_trust_code)+","+\
            str(job.started_on)+","+\
            str(job.started_on_trust_code)+","+\
            str(job.title)+","+\
            str(job.uuid)+","+\
            str(person.uuid)+","+\
            str(job.data['relationships']['organization']['uuid'])+"\n"
        )
    jobs_file.close()
    

### Read all persons

In [602]:
input_file = open('ids_data/people.csv')
input_file.readline()

'name,permalink,updated_at\n'

In [604]:
pattern_people = re.compile('.*,/people/(.*),.*')

In [693]:
count = 0
for line in input_file:
    permalink = pattern_people.match(line).group(1)
    print permalink
    person = cb.person(permalink)
    store_degrees(person)
    store_jobs(person)
    if count>10:
        break
    else:
        count +=1

gabriella-gomez-mont
keith-milburn
tan-choon-shian
marc-casellas
tariq-ali-asghar
maulin-v-shah
nabil-hachicha
stacy-templeton
jihoon-kwon
lawrence-wang-2
kathleen-reed
vineet-raheja
